In [ ]:
import pandas as pd

In [4]:
api_key = "d9632fbcb79e9a8be867cf5a86903ddd355dd4ecdac826d919d4ebc7f3392f35"
secret_key = pd.read_table("paper_future_secret_key.txt").columns[0]

In [ ]:
from binance.client import Client
import pandas as pd

In [ ]:
client = Client(api_key=api_key, api_secret=secret_key, tld="com", testnet=True)

In [ ]:
client.futures_account()

In [ ]:
pd.DataFrame(client.futures_account()["assets"])

In [ ]:
client.futures_account_balance()

In [ ]:
client.futures_position_information(symbol="BTCUSDT")

In [ ]:
client.futures_coin_account()

In [ ]:
client.futures_coin_position_information(symbol="BTCUSD_PERP")

In [ ]:
client.futures_position_information(symbol="BTCUSDT")

In [ ]:
client.futures_get_position_mode()

In [ ]:
client.futures_get_open_orders(symbol="BTCUSDT")

In [ ]:
leverage = 10
size= 0.01

In [ ]:
client.futures_change_leverage(symbol="ETHUSDT", leverage=leverage)

In [ ]:
order_open = client.futures_create_order(symbol="ETHUSDT", side="BUY", type="MARKET", quantity=size)
order_open

In [ ]:
client.futures_get_order(symbol="ETHUSDT", orderId=order_open["orderId"])

In [ ]:
client.futures_position_information(symbol="ETHUSDT")

In [ ]:
order_close = client.futures_create_order(symbol="ETHUSDT", side="SELL", type="MARKET", quantity=0.01, reduce_Only=True)
order_close

In [ ]:
client.futures_get_order(symbol="ETHUSDT", orderId=order_close["orderId"])

In [ ]:
client.futures_position_information(symbol="ETHUSDT")

In [ ]:
client.futures_account_trades(symbol="ETHUSDT")

In [ ]:
df = pd.DataFrame(client.futures_account_trades(symbol="ETHUSDT"))
df.tail(2)

In [ ]:
client.futures_income_history(symbol="ETHUSDT")

In [ ]:
pd.DataFrame(client.futures_income_history(symbol="ETHUSDT"))

In [ ]:
leverage = 15
size = 0.01

In [ ]:
client.futures_change_leverage(symbol="ETHUSDT", leverage=leverage)

In [ ]:
order_open = client.futures_create_order(symbol="ETHUSDT", side="SELL", type="MARKET", quantity=size)
order_open

In [ ]:
client.futures_position_information(symbol="ETHUSDT")

In [ ]:
order_close = client.futures_create_order(symbol="ETHUSDT", side="BUY", type="MARKET", quantity=size, reduceOnly=True)
order_close

In [ ]:
df = pd.DataFrame(client.futures_account_trades(symbol="ETHUSDT"))

In [ ]:
df.tail(2)

In [ ]:
client =

In [ ]:
pd.DataFrame(client.get_historical_klines(symbol="ETHUSDT", interval="1d", start_str="2020-01-01", end_str=None, limit=1000))

In [ ]:
pd.DataFrame(client.futures_historical_klines(symbol="ETHUSDT", interval="1d", start_str="2020-01-01", end_str=None, limit=1000))

In [ ]:
def get_history(symbol, interval, start, end = None):
    bars = client.futures_historical_klines(symbol = symbol, interval = interval,
                                        start_str = start, end_str = end, limit = 1000)
    df = pd.DataFrame(bars)
    df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
    df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                  "Clos Time", "Quote Asset Volume", "Number of Trades",
                  "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
    df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
    df.set_index("Date", inplace = True)
    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors = "coerce")

    return df

In [ ]:
data = get_history(symbol="ETHUSDT", interval="1d", start="2020-01-01")
data

In [ ]:
data.info()

In [ ]:
from binance import ThreadedWebsocketManager
import pandas as pd

In [ ]:
df = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume", "Complete"])
df

In [ ]:
def stream_candles(msg):
    ''' define how to process incoming WebSocket messages '''

    # extract the required items from msg
    event_time = pd.to_datetime(msg["E"], unit = "ms")
    start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
    first   = float(msg["k"]["o"])
    high    = float(msg["k"]["h"])
    low     = float(msg["k"]["l"])
    close   = float(msg["k"]["c"])
    volume  = float(msg["k"]["v"])
    complete=       msg["k"]["x"]

    # print out
    print("Time: {} | Price: {}".format(event_time, close))
    print("test")

    # feed df (add new bar / update latest bar)
    df.loc[start_time] = [first, high, low, close, volume, complete]

In [ ]:
twm = ThreadedWebsocketManager(testnet = True)
twm.start()

In [ ]:
twm.start_kline_futures_socket(callback = stream_candles, symbol = "BTCUSDT", interval = "1m")

In [ ]:
twm.stop()

In [ ]:
df

In [ ]:
df.info()

In [1]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

In [2]:
class FuturesTrader(): # Triple SMA Crossover

    def __init__(self, symbol, bar_length, sma_s, sma_m, sma_l, units, position = 0, leverage = 5):

        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units
        self.position = position
        self.leverage = leverage # NEW
        self.cum_profits = 0 # NEW
        #self.trades = 0
        #self.trade_values = []

        #*****************add strategy-specific attributes here******************
        self.SMA_S = sma_s
        self.SMA_M = sma_m
        self.SMA_L = sma_l
        #************************************************************************

    def start_trading(self, historical_days):

        client.futures_change_leverage(symbol = "BTCUSDT", leverage = self.leverage) # NEW

        self.twm = ThreadedWebsocketManager(testnet = True) # testnet
        self.twm.start()

        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_futures_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length) # Adj: start_kline_futures_socket
        # "else" to be added later in the course

    def get_most_recent(self, symbol, interval, days):

        now = datetime.utcnow()
        past = str(now - timedelta(days = days))

        bars = client.futures_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000) # Adj: futures_historical_klines
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]

        self.data = df

    def stream_candles(self, msg):

        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]

        # print out
        print(".", end = "", flush = True)

        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]

        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades()

    def define_strategy(self):

        data = self.data.copy()

        #******************** define your strategy here ************************
        data = data[["Close"]].copy()

        data["SMA_S"] = data.Close.rolling(window = self.SMA_S).mean()
        data["SMA_M"] = data.Close.rolling(window = self.SMA_M).mean()
        data["SMA_L"] = data.Close.rolling(window = self.SMA_L).mean()

        data.dropna(inplace = True)

        cond1 = (data.SMA_S > data.SMA_M) & (data.SMA_M > data.SMA_L)
        cond2 = (data.SMA_S < data.SMA_M) & (data.SMA_M < data.SMA_L)

        data["position"] = 0
        data.loc[cond1, "position"] = 1
        data.loc[cond2, "position"] = -1
        #***********************************************************************

        self.prepared_data = data.copy()

    def execute_trades(self): # Adj!
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = client.futures_create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade("GOING LONG")
            elif self.position == -1:
                order = client.futures_create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = 2 * self.units)
                self.report_trade("GOING LONG")
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = client.futures_create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade("GOING NEUTRAL")
            elif self.position == -1:
                order = client.futures_create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade("GOING NEUTRAL")
            self.position = 0
        if self.prepared_data["position"].iloc[-1] == -1: # if position is short -> go/stay short
            if self.position == 0:
                order = client.futures_create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade("GOING SHORT")
            elif self.position == 1:
                order = client.futures_create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = 2 * self.units)
                self.report_trade("GOING SHORT")
            self.position = -1

    def report_trade(self, going): # Adj!

        time.sleep(0.1)
        trade = client.futures_account_trades(symbol = "BTCUSDT")[-1]

        # extract data from trade object
        trade_time = pd.to_datetime(trade["time"], unit = "ms")
        price = round(float(trade["price"]), 5)
        base_units = round(float(trade["qty"]), 5)
        quote_units = round(float(trade["quoteQty"]), 5)
        commission = round(-float(trade["commission"]), 5)
        real_profit = round(float(trade["realizedPnl"]), 5)

        # calculate cumulative trading profits
        self.cum_profits += round((commission + real_profit), 5)

        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(trade_time, going))
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(trade_time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(trade_time, real_profit, self.cum_profits))
        print(100 * "-" + "\n")

In [22]:
client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = True)

In [31]:
symbol = "BTCUSDT"
bar_length = "1m"
sma_s = 10
sma_m = 20
sma_l = 50
units = 0.001
position = 0
leverage = 100

In [32]:
trader = FuturesTrader(symbol = symbol, bar_length = bar_length, sma_s = sma_s, sma_m = sma_m, sma_l = sma_l,
                       units = units, position = position, leverage = leverage)

In [54]:
trader.start_trading(historical_days=1/24)

.................................

----------------------------------------------------------------------------------------------------
2023-06-02 12:01:02.147000 | GOING SHORT
2023-06-02 12:01:02.147000 | Base_Units = 0.001 | Quote_Units = 27.4403 | Price = 27440.3 
2023-06-02 12:01:02.147000 | Profit = 0.0 | CumProfits = -0.28958999999999996 
----------------------------------------------------------------------------------------------------

......................................

----------------------------------------------------------------------------------------------------
2023-06-02 12:03:01.122000 | GOING NEUTRAL
2023-06-02 12:03:01.122000 | Base_Units = 0.001 | Quote_Units = 27.448 | Price = 27448.0 
2023-06-02 12:03:01.122000 | Profit = -0.0077 | CumProfits = -0.29732 
----------------------------------------------------------------------------------------------------

................................

----------------------------------------------------------------------

In [55]:
trader.prepared_data

,Close,SMA_S,SMA_M,SMA_L,position
Date,,,,,
2023-06-02 11:49:00,27447.0,27302.35,27317.860,27355.262,-1
2023-06-02 11:50:00,27447.0,27332.02,27332.705,27355.042,-1
2023-06-02 11:51:00,27447.0,27331.02,27332.210,27354.830,-1
2023-06-02 11:52:00,27159.0,27301.23,27317.310,27350.674,-1
2023-06-02 11:53:00,27158.2,27302.00,27317.715,27344.684,-1
2023-06-02 11:54:00,27448.0,27331.75,27317.275,27344.504,0
2023-06-02 11:55:00,27158.2,27302.77,27317.680,27338.528,-1
2023-06-02 11:56:00,27447.7,27302.84,27317.215,27344.272,-1
2023-06-02 11:57:00,27448.0,27331.83,27332.105,27345.232,-1


In [53]:
trader.twm.stop()

.

CANCEL read_loop
